### Calculating Local Field Potentials and Current Source Densities from biophysically detailed network simulations

The goal of this exercise is to explore how to calculate and analyze Local Field Potentials (LFPs) and Current Source Density (CSD) from biophysically detailed neural network simulations. These analyses are critical tools for bridging computational models with experimental neurophysiology.

#### Motivation and Significance
LFPs are among the most commonly recorded signals in neuroscience experiments, providing insight into the collective behavior of neural populations. They represent the extracellular potential generated by the summed electrical activity of many neurons, primarily reflecting synaptic currents and other transmembrane currents. Because the total transmembrane current in a point-neuron model is zero, LFPs can only be computed from biophysically detailed neuron models. In recent years, scientists have worked to develop methods ("tricks") to estimate LFPs from point-neuron models.

CSD analysis transforms LFP measurements into estimates of the net transmembrane current sources and sinks, revealing where ionic currents enter and leave neurons. This provides more precise spatial localization of neural activity than raw LFP recordings.

In this exercise, we'll compare how different synaptic distributions (basal vs. apical) affect the resulting LFPs and CSDs, providing insight into how measured field potentials reflect underlying neural circuit activity.

#### Exercise Overview

Simulate and plot the LFPs and CSD evoked by a reduced L4 mouse V1 network model of 400 $\mu m$ radius consisting of 3 distinct excitatory cell types and an inhibitory PV cell population (180 neurons in total). The network is designed to have a realistic distribution of synaptic connections and neuron types, including basal and apical dendritic synapses. This is a significantly simplified model derived from a more realistic setup to reduce computational time. The network will produce non-smooth LFPs and CSDs that appear simplistic compared to experimental data, unlike the full-scale mouse V1 model. We provide the already built network SONATA files (network*) for a network with basal and apical synapses [network_allsynapses](network_allsynapses), only basal synapses [network_basalsynapses](network_basalsynapses), and only apical synapses [network_apicalsynapses](network_apicalsynapses), respectively. Use `BioNet` to run the simulations of the three networks and compare the evoked LFPs and CSDs.

`Note`: Simulating one configuration/network takes about 15 to 25 minutes on a MacBook Pro.

### Exercise 1. Generate the simulation config.json file:
- The config.json file should include the following parameters:
  - duration: 3 seconds
  - dt: 0.1 ms
  - reports: add 'ecp' recordings to the `reports` section to record the LFPs.
      - `electrode_positions`: path to the csv file containing the electrode positions. File provided in [components/electrodes](components/electrodes).
  - input: use the external input created in step 1.1.2
  - output: save the simulation results in a folder named [outputs](outputs)

`Note`: You can use `config.lfp.json` file as a template. You can modify it to suit your simulation needs.

### Exercise 2. Run the simulation:
- Use `BioNet` to simulate with the generated config.json file. The simulation should be run for 3 seconds with a time step of 0.1 ms. The results should be saved in the outputs folder.
- Remember to compile the [./components/mechanisms](components/mechanisms) before running the simulation. You can do this by running the following command in the terminal:
```bash
! cd components/mechanisms && nrnivmodl modfiles
```

We provide a configuration file `config.lfp.json` that you can use to run the simulation and use as a reference to create the configuration files of the other networks.

### Exercise 3. Plot the results:
- Plot the raster plot of the network activity. The x-axis should represent time, and the y-axis should represent the neuron index. Use different colors for different populations.
- Create a 2D plot of the LFPs recorded by the linear probe. The x-axis should represent time, and the y-axis should represent the electrode number.

```python
from bmtk.analyzer.spike_trains import plot_raster

_ = plot_raster(
    config_file="config.lfp.json",
    spikes_file="output_allsynapses/spikes.csv",
    title="Raster Plot for All Synapses",
    group_by="layer",
)

_ = plot_ecp(config_file="config.lfp_apical.json", report_name="cortical_electrode")
```


### Exercise 4. Calculate the CSD:
- Calculate the CSD from the simulated LFPs using the delta-iCSD method. You can find the implementation of the delta-iCSD method in the [icsd_scripts](icsd_scripts) folder.

In [ ]:
import sys
import quantities as pq
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("icsd_scripts/")

import icsd

from get_csd_input_dict import get_csd_input_dict

delta_icsd_allsynapses = get_csd_input_dict(
    "output_allsynapses/cortical_electrode.h5",
)
delta_icsd_basal = get_csd_input_dict(
    "output_basalsynapses/cortical_electrode.h5",
)
delta_icsd_apical = get_csd_input_dict(
    "output_apicalsynapses/cortical_electrode.h5",
)

In [ ]:
csd_dict = dict(
    delta_icsd=icsd.DeltaiCSD(**delta_icsd_allsynapses),
    delta_icsd_basal=icsd.DeltaiCSD(**delta_icsd_basal),
    delta_icsd_apical=icsd.DeltaiCSD(**delta_icsd_apical),
)

csd_raw = {}
csd_smooth = {}

for method, csd_obj in list(csd_dict.items()):
    csd_raw[method] = csd_obj.get_csd()  # num_channels x trial_duration
    csd_smooth[method] = csd_obj.filter_csd(csd_raw)  # num_channels x trial_duration

In [ ]:
import matplotlib.pyplot as plt

delta_icsd_dict = dict(
    delta_icsd=delta_icsd_allsynapses,
    delta_icsd_basal=delta_icsd_basal,
    delta_icsd_apical=delta_icsd_apical,
)

# for method, csd_smooth in list(csd_smooth.items()):
for method, csd_obj in list(csd_dict.items()):
    csd_raw = csd_obj.get_csd()  # num_channels x trial_duration
    csd_smooth = csd_obj.filter_csd(csd_raw)  # num_channels x trial_duration
    
    fig, axes = plt.subplots(1, 3, figsize=(10, 4))
    lfp_data = delta_icsd_dict[method]["lfp"]

    # plot LFP signal
    ax = axes[0]
    im = ax.imshow(
        np.array(lfp_data),
        origin="upper",
        vmin=-abs(lfp_data).max(),
        vmax=abs(lfp_data).max(),
        cmap="bwr",
    )
    ax.axis(ax.axis("tight"))
    cb = plt.colorbar(im, ax=ax)
    cb.set_label("LFP (%s)" % lfp_data.dimensionality.string)
    ax.set_xticklabels([])
    ax.set_title("LFP")
    ax.set_ylabel("ch #")

    # plot raw csd estimate
    ax = axes[1]
    im = ax.imshow(
        np.array(csd_raw),
        # np.array(csd_raw[method]),
        origin="upper",
        vmin=-abs(csd_raw).max(),
        vmax=abs(csd_raw).max(),
        # vmin=-abs(csd_raw[method]).max(),
        # vmax=abs(csd_raw[method]).max(),
        cmap="jet_r",
    )
    ax.axis(ax.axis("tight"))
    ax.set_title(csd_obj.name)
    cb = plt.colorbar(im, ax=ax)
    cb.set_label("CSD (%s)" % csd_raw.dimensionality.string)
    # cb.set_label("CSD (%s)" % csd_raw[method].dimensionality.string)
    ax.set_xticklabels([])
    ax.set_ylabel("ch #")

    # plot spatially filtered csd estimate
    ax = axes[2]
    im = ax.imshow(
        np.array(csd_smooth),
        origin="upper",
        vmin=-abs(csd_smooth).max(),
        vmax=abs(csd_smooth).max(),
        cmap="jet_r",
    )
    ax.axis(ax.axis("tight"))
    ax.set_title(csd_obj.name + ", filtered")
    cb = plt.colorbar(im, ax=ax)
    cb.set_label("CSD (%s)" % csd_smooth.dimensionality.string)
    ax.set_ylabel("ch #")
    ax.set_xlabel("timestep")